In [1]:
##### 데이터 다운로드
!pip install datasets   # Package install

from datasets import load_dataset   # Huggingface 데이터셋 패키지 import
data = load_dataset("sepidmnorozy/Korean_sentiment")    # 데이터 다운로드

ERROR: Invalid requirement: '#'


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 86.75it/s]


Dataset csv downloaded and prepared to C:/Users/지완/.cache/huggingface/datasets/sepidmnorozy___csv/sepidmnorozy--Korean_sentiment-c4fbcaaf8f2ffcfa/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 76.53it/s]


In [2]:
##### 데이터 구조 훑어보기
# print("Data type: ", type(data))    # 데이터 타입 확인
# print("Data structure: ", data)     # 데이터 구조 확인
# print("Data keys: ", data.keys())   # 데이터 키 확인

print(data['train'][0])   # 실제 데이터 확인

{'label': 1, 'text': '역시 명작 어렸을때 봤을때도 재밌었고 지금 봐도 몇억배 이상으로 재밌어요'}


In [3]:
##### 테스트 세트 만들기
train_data = data['train']
dev_data = data['validation']
test_data = data['test']

#print(train_data)
#print(dev_data)
#print(test_data)

In [4]:
##### 텍스트 데이터 특성
# import matplotlib.pyplot as plt
# plt.hist(data['train']['label'], color='red')
# plt.show()
# plt.hist(data['validation']['label'], color='blue')
# plt.show()
# plt.hist(data['test']['label'], color='green')
# plt.show()

In [5]:
##### 텍스트와 범주형 특성 다루기 (Bag-of-words 방식으로 벡터화 하는 법 구현)
from sklearn.feature_extraction.text import CountVectorizer       # CountVectorizer: Bag of words 벡터화 구현을 하기 위한 클래스

vectorizer = CountVectorizer()      # 데이터를 벡터화 해주는 모델 
vectorizer.fit(train_data['text'])  # 텍스트 문서 모음을 토큰 수의 행렬로 변환

print(vectorizer.vocabulary_)       # 텍스트 문서에 나타난 어휘의 집합을 출력
print(len(vectorizer.vocabulary_))  # 텍스트 문서에 나타난 어휘 집합의 길이를 출력

train_vectors = vectorizer.transform(train_data['text'])    # 학습 데이터를 숫자로 변환
dev_vectors = vectorizer.transform(dev_data['text'])        # 검증 데이터를 숫자로 변환
test_vectors = vectorizer.transform(test_data['text'])      # 테스트 데이터를 숫자로 변환

# print(train_vectors)      # transform 된 결과 확인


##### 텍스트와 범주형 특성 다루기 (Bag-of-words 방식으로 벡터화 한 것 확인하는 방법 구현)
sample_num = -1                            # 확인하고 싶은 샘플 번호
sample_origin = train_data[sample_num]    # 확인하고 싶은 샘플의 원래 문장
sample_transform = train_vectors[sample_num]    # 확인하고 싶은 샘플의 transform된 결과
sample_inverse_transform = vectorizer.inverse_transform(sample_transform) # 확인하고 싶은 샘플의 transform된 결과를 다시 단어의 조합으로 바꾼 결과
print("Original Input:{}\nTransformed: {}\nInv-transformed: {}".format(sample_origin, sample_transform, sample_inverse_transform))    # 출력

{'역시': 61855, '명작': 32925, '어렸을때': 59153, '봤을때도': 42070, '재밌었고': 76577, '지금': 82899, '봐도': 41743, '몇억배': 33163, '이상으로': 70684, '재밌어요': 76545, '이런': 70030, '영화는': 63266, '연습용으로라도': 62377, '만들지말자': 30335, '양심적으로': 58710, '세대': 48335, '우주를': 67029, '넘나드는': 19391, '스카이워커가의': 50549, '비극': 43412, '진짜': 83966, '영화가': 63096, '20여년': 1213, '전에': 77861, '제작되었다는': 79523, '믿기지': 36645, '않는다': 57022, '천재적': 86353, '감독': 6007, '소소한': 48935, '재미로': 75836, '보는': 40380, '영화': 63076, '꽁트들': 15688, '모아서': 33513, '만든': 30037, '느낌': 21064, '이랄까': 69985, 'ㅎㅎ': 4665, '방황하는': 38399, '청춘들아': 86624, '영화를': 63431, '꼭보시길': 15608, '너무': 18955, '느끼한': 21032, '트라볼타': 90980, '음악도': 68987, '별로에': 39786, '스토리도': 50815, '막장': 29787, '믿어봐': 36660, '이건': 69472, '쓰레기야': 54015, 'ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ': 4353, '원작의': 67756, '새로운': 46543, '재해석': 76818, '스토리나': 50802, '특수효과도': 91110, '좋았고': 80975, '전반적으로': 77775, '맘에들었음': 31183, '정말': 78583, '웃긴형이네요ㅋㅋㅋ': 67451, '추천': 87739, '근데': 13481, '99년도에': 2642, '나온걸': 16901, '이제봤네요': 71358, '제가': 

In [6]:
##### 훈련세트에서 훈련하고 평가하기
from sklearn.svm import LinearSVC     #Linear kernel SVM을 사용하기 위한 클래스     

svm = LinearSVC()     # 모델 정의
svm.fit(train_vectors, train_data['label'])     # 모델 학습

print(svm.coef_)      # weights (w)
print(svm.intercept_)     # bias (b)

[[-0.32837313 -0.63274497  0.70058835 ...  0.42035301  0.1197774
  -0.63274497]]
[-0.05088254]


In [7]:
##### 교차 검증을 사용한 평가
# from sklearn.model_selection import cross_val_score

# all_data = train_data['text']+dev_data['text']+test_data['text']      # all_data = train_data + dev_data + test_data
# all_label = train_data['label']+dev_data['label']+test_data['label']  # all_label은 train_data, dev_data, test_date의 레이블을 모두 합한 것 
# all_vectors = vectorizer.transform(all_data)    # all_data를 숫자로 변환
# scores = cross_val_score(svm, all_vectors, all_label, cv=5)  # 5-cross validation

# print("All scores:", scores)  # 전체 점수 출력
# print("Average: {:.2f}%".format(scores.mean()*100)) # 다섯 번의 스코어 평균 출력
# print("Standard deviation: {:.6f}".format(scores.std()))  # 표준편차 출력 

In [8]:
##### 그리드 탐색
# from sklearn.model_selection import GridSearchCV    

# param_grid = [{'max_iter':[500, 1000, 5000], 'C': [1, 10, 100]}]        # 학습 max_iter와 C를 변동 하이퍼파라미터로 설정
# grid_search = GridSearchCV(svm, param_grid, cv=3)     # 하이퍼파라미터 서치 할 모델 설정, 각 모델별로 5-cross validation 실행
# grid_search.fit(train_vectors, train_data['label'])   # transformed train data로 학습

# print(grid_search.cv_results_['mean_test_score'])     # 파라미터 서치 결과
# print(grid_search.best_params_)   #최고의 모델 파라미터 

# #파라미터 서치 결과 출력
# for mean_score, params in zip(grid_search.cv_results_['mean_test_score'], grid_search.cv_results_['params']):
#   print(mean_score, params)       

In [9]:
##### 테스트 세트로 시스템 평가하기
from sklearn.metrics import accuracy_score    # Accuracy 측정 함수 import

final_model = svm #grid_search.best_estimator_     # grid search에 의해 정해진 제일 좋은 하이퍼파라미터

pred_results = final_model.predict(test_vectors)    # 최종 모델로 test 데이터 예측
accuracy = accuracy_score(test_data['label'], pred_results)   # 정확도 측정

print("Accuracy: {:.2f}%".format(accuracy*100))   # 정확도 출력 

Accuracy: 79.34%
